# $k$-Nearest Neighbors (KNN)

## Objectives

- Describe the $k$-nearest neighbors algorithm
- Identify multiple common distance metrics
- Tune $k$ appropriately in response to models with high bias or variance

## Who's Nearby?

One strategy to make predictions on a new data point is to just look at what _similar_ data points are classified as.

### So: What should the grey point be?

![example scenario](images/scenario.png)

## KNN has the following basic steps:

![knn process](images/knn-process.png)

**The algorithm can be summarized as:**

1. A positive integer `k` is specified
2. We select the `k` entries in our training data which are closest to the new sample
3. We find the most common classification of these entries (voting)
4. This is the classification we give to the new sample

**A few other features of KNN:**

* KNN stores the entire training dataset which it uses as its representation
* KNN does not learn any model parameters
* KNN makes predictions 'just-in-time' by calculating the similarity between an input sample and each training instance

**Note:** KNN performs better with a low number of features. The more features you have the more data you need. You increase the dimensions everytime you add another feature. Increase in dimension also leads to the problem of overfitting. To avoid overfitting, the needed data will need to grow exponentially as you increase the number of dimensions. This problem of higher dimension is known as the Curse of Dimensionality.

Resource on the Curse of Dimensionality for classification: https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/

### Voting

How to break ties:

1. When doing a binary classification, often use a odd `k` to avoid ties.
2. Multiple approaches for multi-class problems:
    - Reduce the `k` by 1 to see who wins.
    - Weight the votes based on the distance of the neighbors

## Distance Metrics

When using KNN, we're assuming that _nearby_ points are _similar_ to one another. There are a few different wasy to determine how "close" data points are to one another:

Distance metrics give us formulas to measure the distance between our variables:

![](https://miro.medium.com/max/1400/1*FlMiuoENrq52tMV4S6LSZg.png)

> Euclidean and Manhattan distance are typically best for continuous variables 

## Implementing in Scikit-Learn

Let's try doing some basic classification on some data using the KNN algorithms.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, recall_score, precision_score

In [ ]:
# Same processed version of recent Austin Animal Shelter data
# Goal is to predict whether an animal is adopted
df = pd.read_csv("data/processed_shelter.csv")
df.head()

In [ ]:
X = df.drop(['adoption'], axis=1)
y = df.adoption

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=.2)
X_train.head()

#### Should We Scale?

KNN IS a linear model - you **NEED** to scale your data before fitting a k-nearest neighbors model!

KNN uses distances to determine neighbors, so it's comparing across features - it needs to have a consistent understanding of scale in order to do this appropriately!

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
# Instantiate our KNN classifier - can use default k=5
knn = None

print(f"Train Accuracy: {knn.score(X_train_sc, y_train)}")
print(f"Test Accuracy: {knn.score(X_test_sc, y_test)}")

In [ ]:
# Evaluate
test_preds = knn.predict(X_test_sc)

print(f'Test Accuracy Score: {accuracy_score(y_test, test_preds)}')
print('-------------------')
print(f'Test Precision Score: {precision_score(y_test, test_preds)}')
print('-------------------')
print(f'Test Recall Score: {recall_score(y_test, test_preds)}')
print('-------------------')
print(f'Test F1_score: {f1_score(y_test, test_preds)}')
print('-------------------')
# Note the different input for ROC-AUC!
print(f'Test ROC-AUC Score: {roc_auc_score(y_test, knn.predict_proba(X_test_sc)[:,1])}')

In [ ]:
plot_confusion_matrix(knn, X_test_sc, y_test, display_labels=['not adopted', 'adopted']);

## Different $k$ Values

A big factor in this algorithm is choosing $k$

# $k$ and the Bias-Variance Tradeoff

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    test_size=0.25)

In [ ]:
# Let's slowly increase k and see what happens to our accuracy scores.

from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

k_scores_train = {}
k_scores_val = {}


for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    accuracy_score_t = []
    accuracy_score_v = []
    for train_ind, val_ind in kf.split(X_train, y_train):
        
        X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind] 
        X_v, y_v = X_train.iloc[val_ind], y_train.iloc[val_ind]
        mm = MinMaxScaler()
        
        X_t_ind = X_t.index
        X_v_ind = X_v.index
        
        X_t = pd.DataFrame(mm.fit_transform(X_t))
        X_t.index = X_t_ind
        X_v = pd.DataFrame(mm.transform(X_v))
        X_v.index = X_v_ind
        
        knn.fit(X_t, y_t)
        
        y_pred_t = knn.predict(X_t)
        y_pred_v = knn.predict(X_v)
        
        accuracy_score_t.append(accuracy_score(y_t, y_pred_t))
        accuracy_score_v.append(accuracy_score(y_v, y_pred_v))
        
        
    k_scores_train[k] = np.mean(accuracy_score_t)
    k_scores_val[k] = np.mean(accuracy_score_v)

In [ ]:
k_scores_train

In [ ]:
k_scores_val

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(list(k_scores_train.keys()), list(k_scores_train.values()),
        color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10, label='Train')
ax.plot(list(k_scores_val.keys()), list(k_scores_val.values()),
        color='green', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10, label='Val')
ax.set_xlabel('k')
ax.set_ylabel('Accuracy')
ax.set_xticks(ticks=list(k_scores_val.keys()))
plt.legend();

### Discuss:

- 


## The Relation Between $k$ and Bias/Variance

> Small $k$ values leads to overfitting, but larger $k$ values tend towards underfitting

<img src="images/K-NN_Neighborhood_Size_print.png" width=600>

> From [Machine Learning Flashcards](https://machinelearningflashcards.com/) by Chris Albon

# Summary

K-Nearest Neighbors is a **non-parametric**, **lazy** learning algorithm. 

What does this mean?

- **Non-parametric models** make no underlying assumptions about the distribution of data
- **Lazy learners** (or **instance-based** learning methods) simply store the training examples and postpone the generalization (building a model) until a new instance must be classified or prediction made
    - In other words, no training is necessary! This makes training super fast but testing is slower and costly

## Pros

- Lazy learning (no training phase)
- Simple algorithm to understand and implement

## Cons

- Computationally expensive
    - It doesn't learn anything when the model is `fit` - it just stores all the training data points in memory
    - Thus: high memory requirement, prediction stage can be slow
- Not robust - doesn't generalize well
- Soft boundaries are troublesome
- "Curse of Dimensionality"
    - Sensitive to irrelevant features and the scale of data
    - Thus: doesn't work well with a lot of input features

# Additional Resources

- [Nearest Neighbors](http://scikit-learn.org/stable/modules/neighbors.html) (user guide), [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) (class documentation)

- [Videos from An Introduction to Statistical Learning](http://www.dataschool.io/15-hours-of-expert-machine-learning-videos/)
    - Classification Problems and K-Nearest Neighbors (Chapter 2)
    - Introduction to Classification (Chapter 4)
    - Logistic Regression and Maximum Likelihood (Chapter 4)
    
- [Curse of Dimensionality](https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/)

### More Resources on Scaling:

https://sebastianraschka.com/Articles/2014_about_feature_scaling.html   
http://datareality.blogspot.com/2016/11/scaling-normalizing-standardizing-which.html

# Bonus: Visualize the Decision Boundaries

One way to think about KNN is that there are decision boundaries, areas in which we know exactly what the prediction will be for some given `k`, since we can calculate how many points of each class are nearby. SKLearn and matplotlib can help us visualize this!

Reference: https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html

In [ ]:
# Let's read in the Chemical Manufacturing Process data
chem_df = pd.read_csv("data/ChemicalManufacturingProcess.csv", index_col=0)
chem_df.head()

In [ ]:
used_cols = ['BiologicalMaterial01', 'BiologicalMaterial02']
X = chem_df[used_cols]
y = (chem_df['Yield'] >= 41).astype(int)

In [ ]:
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

# Can adjust our number of neighbors to explore
n_neighbors = 5
h = .02  # step size in the mesh

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X.values[:, 0].min() - 1, X.values[:, 0].max() + 1
    y_min, y_max = X.values[:, 1].min() - 1, X.values[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X.values[:, 0], X.values[:, 1], c=y, cmap=cmap_bold)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Binary Classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))

plt.show()

# Level Up: Distance Metrics

> The "closeness" of data points → proxy for similarity

![](img/distances.png)

**Minkowski Distance**:

$$dist(A,B) = (\sum_{k=1}^{N} |a_k - b_k|^c)^\frac{1}{c} $$

Special cases of Minkowski distance are:

- Manhattan: $dist(A,B) = \sum_{k=1}^{N} |a_k - b_k|$


- Euclidean: $dist(A,B) = \sqrt{ \sum_{k=1}^{N} (a_k - b_k)^2 }$

There are quite a few different distance metrics built-in for Scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html 